In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from google.colab import drive

drive.mount('/content/drive')
!pip install python-dotenv
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv('/content/drive/MyDrive/GIVORA.io/.env')

# Retrieve API credentials from environment variables
API_KEY = os.getenv("API_KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH_ENGINE_ID")
def google_search(query,key,cx,num_results):
  url = "https://www.googleapis.com/customsearch/v1"
  params = {
      "key": API_KEY,
      "cx": SEARCH_ENGINE_ID,
      "q": query,
       "num": num_results  # maximum is 10 per request
  }
  response = requests.get(url, params=params)
  if response.status_code == 200:
      data = response.json()
      return data.get("items", [])

  else:
      print("Error:", response.status_code)
      return []

Mounted at /content/drive


In [2]:
# prompt: ask and let user upload only pdf and doc

from google.colab import files
import os

def upload_pdf_or_doc():
  uploaded = files.upload()
  for fn in uploaded.keys():
    if not (fn.lower().endswith('.pdf') or fn.lower().endswith('.doc') or fn.lower().endswith('.docx')):
      print(f"Error: Invalid file type for '{fn}'. Only PDF and DOC/DOCX files are allowed.")
      # Optionally, remove the invalid file:
      os.remove(fn)
      return None  # Indicate failure
    else:
      print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
      return fn #return the valid filename

# Example usage
filename = upload_pdf_or_doc()
if filename:
    print(f"Successfully uploaded {filename}")


Saving Rezume.pdf to Rezume.pdf
User uploaded file "Rezume.pdf" with length 39434 bytes
Successfully uploaded Rezume.pdf


In [3]:
# prompt: NLP to read the  file

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def analyze_file(filename):
    try:
        with open(filename, 'r') as file:
            text = file.read()
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Perform part-of-speech tagging
    pos_tags = nltk.pos_tag(tokens)

    # Perform named entity recognition
    named_entities = nltk.ne_chunk(pos_tags)

    # Print the results
    print("Tokens:", tokens)
    print("POS Tags:", pos_tags)
    print("Named Entities:", named_entities)


# Example usage (assuming 'filename' is defined from the previous code)
if 'filename' in locals() and filename:  # Check if filename is defined and not None
    analyze_file(filename)
else:
    print("No valid filename provided. Please upload a file first.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 10: invalid start byte

In [ ]:

# import csv
# def save_to_csv(data, filename="search_results.csv"):
#     with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
#         fieldnames = ['title', 'link', 'snippet'] # Define the fields you want to save
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()

#         for item in data:
#           try:
#             writer.writerow({
#                 'title': item['title'],
#                 'link': item['link'],
#                 'snippet': item['snippet']
#             })
#           except KeyError as e:
#             print(f"Warning: Skipping an item due to missing key {e}")

search_results = google_search("Jobs Website", key=API_KEY, cx=SEARCH_ENGINE_ID, num_results=10)
for item in search_results:
    print("Title:", item['title'])
    print("Link:", item['link'])
    print("Snippet:", item['snippet'])
    print("-" * 20)

Title: USAJOBS - The Federal Government's official employment site
Link: https://www.usajobs.gov/
Snippet: Explore career fields that are hiring now. The government is looking to fill positions that are in high demand. Find your opportunity to shape America's future.
--------------------
Title: LinkedIn Job Search: Find US Jobs, Internships, Jobs Near Me
Link: https://www.linkedin.com/jobs
Snippet: ... of job seekers get hired through a referral. Use LinkedIn Jobs to boost your chances of getting hired through people you know ... Web Design 560+ courses ...
--------------------
Title: Monster Jobs - Job Search, Career Advice & Hiring Resources ...
Link: https://www.monster.com/
Snippet: Monster is your source for jobs and career opportunities. Search for jobs ... Online Teaching Jobs · Medical Assistant Jobs · Delivery Driver Jobs · Security ...
--------------------
Title: Job Search: Millions of US Jobs (HIRING NOW) Near You - 2025
Link: https://www.ziprecruiter.com/
Snippet: Search R

In [ ]:
# industry_keywords = {
#         "Technology": ["developer", "engineer", "software", "IT", "AI", "cloud"],
#         "Healthcare": ["nurse", "doctor", "medical", "hospital"],
#         "Finance": ["accountant", "finance", "banking", "investment"],
#         "Retail": ["store", "sales", "retail"],
#         "Manufacturing": ["factory", "production", "assembly"],
#         "Education": ["teacher", "professor", "lecturer"],
#         "Construction": ["civil engineer", "construction", "site manager"],
#         "Transportation": ["driver", "logistics", "transport"],
#         "Hospitality": ["hotel", "chef", "restaurant"],
#         "Media & Entertainment": ["journalist", "actor", "musician"],
#         "Government": ["public sector", "policy", "administration"],
#         "Non-Profit": ["ngo", "volunteer", "charity"],
#         "Legal": ["lawyer", "legal", "attorney"],
#         "Engineering": ["mechanical engineer", "electrical engineer"],
#         "Sales": ["sales executive", "business development"],
#         "Marketing": ["marketing", "SEO", "advertising"],
#         "Customer Service": ["customer service", "support"],
#         "Agriculture": ["farm", "agriculture", "fisheries"],
#         "Consulting": ["consultant", "business analyst"],
#     }

import pandas as pd

df = pd.DataFrame(search_results)
df

,kind,title,htmlTitle,link,displayLink,snippet,htmlSnippet,formattedUrl,htmlFormattedUrl,pagemap
0,customsearch#result,USAJOBS - The Federal Government's official em...,USAJOBS - The Federal Government&#39;s officia...,https://www.usajobs.gov/,www.usajobs.gov,Explore career fields that are hiring now. The...,Explore career fields that are hiring now. The...,https://www.usajobs.gov/,https://www.usa<b>jobs</b>.gov/,{'metatags': [{'viewport': 'width=device-width...
1,customsearch#result,"LinkedIn Job Search: Find US Jobs, Internships...",LinkedIn <b>Job</b> Search: Find US <b>Jobs</b...,https://www.linkedin.com/jobs,www.linkedin.com,... of job seekers get hired through a referra...,... of <b>job</b> seekers get hired through a ...,https://www.linkedin.com/jobs,https://www.linkedin.com/<b>jobs</b>,{'cse_thumbnail': [{'src': 'https://encrypted-...
2,customsearch#result,"Monster Jobs - Job Search, Career Advice & Hir...","Monster <b>Jobs</b> - <b>Job</b> Search, Caree...",https://www.monster.com/,www.monster.com,Monster is your source for jobs and career opp...,Monster is your source for <b>jobs</b> and car...,https://www.monster.com/,https://www.monster.com/,{'cse_thumbnail': [{'src': 'https://encrypted-...
3,customsearch#result,Job Search: Millions of US Jobs (HIRING NOW) N...,<b>Job</b> Search: Millions of US <b>Jobs</b> ...,https://www.ziprecruiter.com/,www.ziprecruiter.com,Search Rated #1 job site in the US * Based on ...,Search Rated #1 <b>job site</b> in the US * Ba...,https://www.ziprecruiter.com/,https://www.ziprecruiter.com/,{'cse_thumbnail': [{'src': 'https://encrypted-...
4,customsearch#result,Indeed: Job Search,Indeed: <b>Job</b> Search,https://www.indeed.com/,www.indeed.com,"With Indeed, you can search millions of jobs o...","With Indeed, you can search millions of <b>job...",https://www.indeed.com/,https://www.indeed.com/,{'cse_thumbnail': [{'src': 'https://encrypted-...
5,customsearch#result,Glassdoor | Job Search & Career Community,Glassdoor | <b>Job</b> Search &amp; Career Com...,https://www.glassdoor.com/index.htm,www.glassdoor.com,Find your work people at the new home for work...,Find your work people at the new home for work...,https://www.glassdoor.com/index.htm,https://www.glassdoor.com/index.htm,{'cse_thumbnail': [{'src': 'https://encrypted-...
6,customsearch#result,OhioMeansJobs | Ohio.gov,OhioMeansJobs | Ohio.gov,https://ohiomeansjobs.ohio.gov/,ohiomeansjobs.ohio.gov,... website belongs to an official government ...,... <b>website</b> belongs to an official gove...,https://ohiomeansjobs.ohio.gov/,https://ohiomeans<b>jobs</b>.ohio.gov/,{'cse_thumbnail': [{'src': 'https://encrypted-...
7,customsearch#result,CalCareers,CalCareers,https://calcareers.ca.gov/,calcareers.ca.gov,A staff member will be available to assist you...,A staff member will be available to assist you...,https://calcareers.ca.gov/,https://calcareers.ca.gov/,"{'metatags': [{'handheldfriendly': 'True', 'vi..."
8,customsearch#result,Find a Job | Department of Labor,Find a <b>Job</b> | Department of Labor,https://dol.ny.gov/find-job-0,dol.ny.gov,Using New York's Job Bank you can search open ...,Using New York&#39;s Job Bank you can search o...,https://dol.ny.gov/find-job-0,https://dol.ny.gov/find-job-0,{'cse_thumbnail': [{'src': 'https://encrypted-...
9,customsearch#result,Department of Workforce Services,Department of Workforce Services,https://jobs.utah.gov/,jobs.utah.gov,"Search our robust job board of more than 35,00...",Search our robust <b>job</b> board of more tha...,https://jobs.utah.gov/,https://<b>jobs</b>.utah.gov/,{'cse_thumbnail': [{'src': 'https://encrypted-...


In [ ]:
def extract_job_details(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raises an HTTPError if the response was an error
        soup = BeautifulSoup(response.content, "html.parser")

        # Example extraction - customize for each site
        job_title_elem = soup.find("h1", class_="job-title")
        job_desc_elem = soup.find("div", id="job-description")
        job_date_elem = soup.find("span", class_="job-date")
        job_location_elem = soup.find("div", class_="job-location")
        employer_elem = soup.find("div", class_="employer-name")

        job_title = job_title_elem.text.strip() if job_title_elem else "N/A"
        job_description = job_desc_elem.text.strip() if job_desc_elem else "N/A"
        job_date = job_date_elem.text.strip() if job_date_elem else "N/A"
        job_location = job_location_elem.text.strip() if job_location_elem else "N/A"
        employer = employer_elem.text.strip() if employer_elem else "N/A"

        return {
            "job_title": job_title,
            "job_description": job_description,
            "job_date": job_date,
            "job_location": job_location,
            "employer": employer
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        return None

# Iterating through your DataFrame's 'link' column
job_data = []
for index, row in df.iterrows():
    link = row['link']
    job_details = extract_job_details(link)
    if job_details:
        job_details['link'] = link
        job_data.append(job_details)

jobs_df = pd.DataFrame(job_data)
print(jobs_df)


Error fetching URL https://www.indeed.com/: 403 Client Error: Forbidden for url: https://www.indeed.com/
Error fetching URL https://www.glassdoor.com/index.htm: 403 Client Error: Forbidden for url: https://www.glassdoor.com/index.htm
  job_title job_description job_date job_location employer  \
0       N/A             N/A      N/A          N/A      N/A   
1       N/A             N/A      N/A          N/A      N/A   
2       N/A             N/A      N/A          N/A      N/A   
3       N/A             N/A      N/A          N/A      N/A   
4       N/A             N/A      N/A          N/A      N/A   
5       N/A             N/A      N/A          N/A      N/A   
6       N/A             N/A      N/A          N/A      N/A   
7       N/A             N/A      N/A          N/A      N/A   

                              link  
0         https://www.usajobs.gov/  
1    https://www.linkedin.com/jobs  
2         https://www.monster.com/  
3    https://www.ziprecruiter.com/  
4  https://ohiomeansjo

In [ ]:
import csv
import requests
from googlesearch import search

def google_search(query, num_results=10):
    results = []
    for url in search(query, num=num_results):
        results.append(url)
    return results

def save_links_to_csv(links, filename="job_boards.csv"):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Job Board Links"])
        for link in links:
            writer.writerow([link])

job_board_links = google_search("Job board site", num_results=10)
save_links_to_csv(job_board_links)


In [ ]:
import time
import random
from requests.exceptions import RequestException

def scrape_jobs(url, max_retries=3, delay_range=(1, 3)):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }

    retry_count = 0
    while retry_count < max_retries:
        try:
            # Add random delay to avoid detection
            time.sleep(random.uniform(*delay_range))

            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                return parse_job_listings(soup, url)
            elif response.status_code == 403 or response.status_code == 429:
                print(f"Rate limited on {url}, waiting longer before retry...")
                time.sleep(random.uniform(5, 10))  # Longer wait for rate limiting
            else:
                print(f"Failed to access {url} - Status code: {response.status_code}")

        except RequestException as e:
            print(f"Error accessing {url}: {e}")

        retry_count += 1

    print(f"Max retries exceeded for {url}")
    return []

In [ ]:
def parse_job_listings(soup, url):
    if "indeed.com" in url:
        return parse_indeed(soup)
    elif "linkedin.com" in url:
        return parse_linkedin(soup)
    elif "monster.com" in url:
        return parse_monster(soup)
    else:
        return parse_generic(soup)

def parse_indeed(soup):
    jobs = []
    job_cards = soup.find_all('div', class_='jobsearch-ResultsList')

    for card in job_cards:
        job_elements = card.find_all('div', class_='job_seen_beacon')
        for job in job_elements:
            title_elem = job.find('h2', class_='jobTitle')
            company_elem = job.find('span', class_='companyName')
            location_elem = job.find('div', class_='companyLocation')

            title = title_elem.text.strip() if title_elem else "N/A"
            company = company_elem.text.strip() if company_elem else "N/A"
            location = location_elem.text.strip() if location_elem else "N/A"
            link = "https://indeed.com" + job.find('a')['href'] if job.find('a') else "N/A"

            jobs.append({
                'title': title,
                'company': company,
                'location': location,
                'link': link
            })

    return jobs

# Similarly implement parse_linkedin, parse_monster, etc.

In [ ]:
import sqlite3

def initialize_database():
    conn = sqlite3.connect('job_database.db')
    cursor = conn.cursor()

    # Create tables
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS job_listings (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        company TEXT NOT NULL,
        location TEXT,
        description TEXT,
        link TEXT UNIQUE,
        category TEXT,
        date_posted TEXT,
        date_scraped TEXT,
        source TEXT
    )
    ''')

    conn.commit()
    return conn, cursor

def save_jobs_to_database(jobs, source):
    conn, cursor = initialize_database()
    current_date = time.strftime('%Y-%m-%d')

    for job in jobs:
        try:
            cursor.execute('''
            INSERT OR IGNORE INTO job_listings
            (title, company, location, link, category, date_scraped, source)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                job['title'],
                job['company'],
                job['location'],
                job['link'],
                job.get('category', 'Uncategorized'),
                current_date,
                source
            ))
        except sqlite3.Error as e:
            print(f"Database error: {e}")

    conn.commit()
    conn.close()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

def train_job_categorizer(training_data):
    """
    Train a simple job categorizer using Naive Bayes

    training_data: List of dictionaries with 'text' and 'category' keys
    """
    texts = [item['text'] for item in training_data]
    categories = [item['category'] for item in training_data]

    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(texts)

    classifier = MultinomialNB()
    classifier.fit(X, categories)

    return vectorizer, classifier

def categorize_job(job_title, job_description, vectorizer, classifier):
    """Categorize a job based on its title and description"""
    text = job_title + " " + (job_description or "")
    features = vectorizer.transform([text])
    category = classifier.predict(features)[0]
    return category

In [ ]:
def get_proxy():
    # This could be a list of proxies you maintain or from a proxy service
    proxy_list = [
        "http://proxy1.example.com:8080",
        "http://proxy2.example.com:8080",
        "http://proxy3.example.com:8080"
    ]
    return random.choice(proxy_list)

def scrape_with_proxy(url, max_retries=3):
    for _ in range(max_retries):
        proxy = get_proxy()
        proxies = {
            "http": proxy,
            "https": proxy
        }

        try:
            response = requests.get(
                url,
                headers=get_headers(),
                proxies=proxies,
                timeout=10
            )
            if response.status_code == 200:
                return response
        except Exception as e:
            print(f"Error with proxy {proxy}: {e}")
            continue

    return None

In [ ]:
import nest_asyncio
import asyncio
import aiohttp
from bs4 import BeautifulSoup

# Apply nest_asyncio to allow running async code in Jupyter/Colab
nest_asyncio.apply()

async def fetch_url(session, url):
    try:
        async with session.get(url, headers=get_headers()) as response:
            if response.status == 200:
                return await response.text()
            else:
                print(f"Error: {url} returned {response.status}")
                return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

async def scrape_multiple_sites_async(urls):
    results = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(fetch_url(session, url))

        responses = await asyncio.gather(*tasks)

        for url, response in zip(urls, responses):
            if response:
                soup = BeautifulSoup(response, 'html.parser')
                jobs = parse_job_listings(soup, url)
                results.extend(jobs)

    return results

# Helper function for headers
def get_headers():
    return {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }

# Usage in Jupyter/Colab environment
job_board_links = ["https://www.indeed.com/"]

# Instead of asyncio.run(), use this approach:
loop = asyncio.get_event_loop()
results = loop.run_until_complete(scrape_multiple_sites_async(job_board_links))

Error: https://www.indeed.com/ returned 403


In [ ]:
def categorize_jobs(jobs):
    industry_keywords = {
        "Technology": ["developer", "engineer", "software", "IT", "AI", "cloud"],
        "Healthcare": ["nurse", "doctor", "medical", "hospital"],
        "Finance": ["accountant", "finance", "banking", "investment"],
        "Retail": ["store", "sales", "retail"],
        "Manufacturing": ["factory", "production", "assembly"],
        "Education": ["teacher", "professor", "lecturer"],
        "Construction": ["civil engineer", "construction", "site manager"],
        "Transportation": ["driver", "logistics", "transport"],
        "Hospitality": ["hotel", "chef", "restaurant"],
        "Media & Entertainment": ["journalist", "actor", "musician"],
        "Government": ["public sector", "policy", "administration"],
        "Non-Profit": ["ngo", "volunteer", "charity"],
        "Legal": ["lawyer", "legal", "attorney"],
        "Engineering": ["mechanical engineer", "electrical engineer"],
        "Sales": ["sales executive", "business development"],
        "Marketing": ["marketing", "SEO", "advertising"],
        "Customer Service": ["customer service", "support"],
        "Agriculture": ["farm", "agriculture", "fisheries"],
        "Consulting": ["consultant", "business analyst"],
    }

    categorized_jobs = {industry: [] for industry in industry_keywords.keys()}

    for job in jobs:
        job_title = job['title'].lower()
        for industry, keywords in industry_keywords.items():
            if any(keyword in job_title for keyword in keywords):
                categorized_jobs[industry].append(job)
                break  # Assign to the first matching industry

    return categorized_jobs

categorized_jobs = categorize_jobs(job_listings)


NameError: name 'job_listings' is not defined